In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
import json
import time
from pprint import pprint

In [13]:
# Load dataset
with open('refined_tv_data_tech_test_v3.json', 'r') as f:
    data = json.load(f)
print("size of dataset: ", len(data))

all_tags = set()
all_title = set()
for item in data:
    if 'tags' in item:
        all_tags.update(list(item['tags']))
    if 'defaultTitle' in item:
        all_title.add(item['defaultTitle'])


print(f"All {len((list(all_tags)))} unique tags:")
pprint(list(all_tags))

print(f"\nThere are {len(list(all_title))} unique titles.")


size of dataset:  50000
All 33 unique tags:
['jeunes_et_ados',
 'jeux_video',
 'magazine_sportif',
 'film_et_serie_d_auteur',
 'crime',
 'fantastique',
 'action',
 'football_championnat_it',
 'meurtre',
 'basketball',
 'chevaux',
 'sport_mecanique',
 'thrillers',
 'sports_feminins',
 'enquete',
 'football',
 'comedies',
 'films_et_series_2020s',
 'football_championnat_port',
 'sport_de_glisse',
 'drames',
 'films_et_series_2010s',
 'combat',
 'attirance_et_sentiments',
 'musiciens',
 'relations_familiales',
 'football_championnat_uk',
 'sport_d_hiver',
 'sport_de_raquette',
 'football_championnat_fr',
 'groupe_d_amis',
 'rugby',
 'police']

There are 6745 unique titles.


In [14]:
df = pd.DataFrame(data)

# Combine list of strings into one string
def list_to_string(list_of_strings):
    if isinstance(list_of_strings, list):
      list_of_strings = [s.strip() for s in list_of_strings]
      list_of_strings = [s.replace('"', '').replace('«', '').replace('»', '') for s in list_of_strings]
      list_of_strings = list(dict.fromkeys(list_of_strings))
      return ' '.join(list_of_strings)
    return ''

df['synopsis'] = df['synopsis'].apply(list_to_string)

In [15]:
# Combine title + synopsis as input text
df["text"] = df["defaultTitle"] + " " + df["synopsis"]

# ----------------------------
# 2. Preprocess labels (multi-label binarizer)
# ----------------------------
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df["tags"])

# ----------------------------
# 3. Feature extraction (TF-IDF)
# ----------------------------
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["text"])

# ----------------------------
# 4. Train/test split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Linear regression

In [32]:
# ----------------------------
# 5. Train multi-label classifier
# ----------------------------
start = time.perf_counter()
clf = OneVsRestClassifier(LogisticRegression(max_iter=1000))
clf.fit(X_train, y_train)
print("Training took:", time.perf_counter() - start, "seconds")

# ----------------------------
# 6. Evaluate
# ----------------------------
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=0))

Training took: 1.4519792499777395 seconds
                           precision    recall  f1-score   support

                   action       1.00      0.01      0.02        82
  attirance_et_sentiments       0.45      0.03      0.05       189
               basketball       1.00      0.97      0.98       878
                  chevaux       1.00      0.93      0.96       350
                   combat       1.00      0.97      0.99      2024
                 comedies       0.88      0.11      0.19       266
                    crime       0.87      0.33      0.47       374
                   drames       0.00      0.00      0.00       108
                  enquete       0.97      0.18      0.31       212
              fantastique       0.91      0.05      0.10       190
   film_et_serie_d_auteur       0.60      0.04      0.07       157
    films_et_series_2010s       0.00      0.00      0.00       202
    films_et_series_2020s       0.33      0.00      0.01       338
                 fo

### Support vector machine

In [ ]:
# ----------------------------
# 5. Train multi-label classifier
# ----------------------------
start = time.perf_counter()
clf = OneVsRestClassifier(SVC())
clf.fit(X_train, y_train)
print("Training took:", time.perf_counter() - start, "seconds")

# ----------------------------
# 6. Evaluate
# ----------------------------
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=0))

### Random forest

In [31]:
# ----------------------------
# 5. Train multi-label classifier
# ----------------------------
start = time.perf_counter()
clf = OneVsRestClassifier(RandomForestClassifier(max_depth=None, random_state=0))
clf.fit(X_train, y_train)
print("Training took:", time.perf_counter() - start, "seconds")

# ----------------------------
# 6. Evaluate
# ----------------------------
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=0))

Training took: 131.97654004200012 seconds
                           precision    recall  f1-score   support

                   action       1.00      0.04      0.07        82
  attirance_et_sentiments       1.00      0.01      0.01       189
               basketball       1.00      0.99      1.00       878
                  chevaux       1.00      0.96      0.98       350
                   combat       1.00      0.99      0.99      2024
                 comedies       1.00      0.04      0.07       266
                    crime       0.91      0.29      0.44       374
                   drames       0.00      0.00      0.00       108
                  enquete       0.89      0.16      0.27       212
              fantastique       1.00      0.05      0.10       190
   film_et_serie_d_auteur       1.00      0.01      0.03       157
    films_et_series_2010s       0.86      0.03      0.06       202
    films_et_series_2020s       0.94      0.04      0.08       338
                 fo

In [6]:
import joblib

# Save model and vectorizer
# joblib.dump(clf, "tagging_model.pkl")
# joblib.dump(vectorizer, "vectorizer.pkl")
# joblib.dump(mlb, "label_binarizer.pkl")

# Load later
clf = joblib.load("tagging_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")
mlb = joblib.load("label_binarizer.pkl")

In [28]:
# ----------------------------
# 7. Inference: predict tags for a new video
# ----------------------------
new_videos = ["Liga Portugal : William Gomes et le FC Porto continuent leur parcours parfait",
             "L'engouement pour la WNBA",
              "Champions Cup: le Stade Toulousain décimé, l'occasion rêvée pour l'UBB",
              "Champions Cup: le Stade Toulousain pour l'UBB",
              "A Philly, Guerschon Yabusele",
              "Lilo & Stitch, L’histoire touchante et drôle d’une petite fille hawaïenne solitaire et d’un extra-terrestre fugitif qui l’aide à renouer le lien avec sa famille.",
              "Freaky Friday dans la peau de ma mère, Veuve sur le point de se remarier, Tess et sa fille Anna ne s'entendent pas. Un jeudi, leur rancoeur éclate. Deux biscuits vont tout compliquer en créant un choc mystique. Le lendemain, Tess et Anna se retrouvent dans le corps l'une de l'autre...",
              "Astérix et Obélix : Mission Cléopâtre, Cléopâtre décide, pour défier l'Empereur romain Jules César, de construire en trois mois un palais somptueux en plein désert. Si elle y parvient, celui-ci devra concéder que le peuple égyptien est le plus grand de tous les peuples...",]
X_new = vectorizer.transform(new_videos)
preds = clf.predict(X_new)
predicted_tags = mlb.inverse_transform(preds)
print("New videos:")
pprint(new_videos)
print("\nSuggested tags:")
pprint(predicted_tags)

New videos:
['Liga Portugal : William Gomes et le FC Porto continuent leur parcours '
 'parfait',
 "L'engouement pour la WNBA",
 "Champions Cup: le Stade Toulousain décimé, l'occasion rêvée pour l'UBB",
 "Champions Cup: le Stade Toulousain pour l'UBB",
 'A Philly, Guerschon Yabusele',
 'Lilo & Stitch, L’histoire touchante et drôle d’une petite fille hawaïenne '
 'solitaire et d’un extra-terrestre fugitif qui l’aide à renouer le lien avec '
 'sa famille.',
 'Freaky Friday dans la peau de ma mère, Veuve sur le point de se remarier, '
 "Tess et sa fille Anna ne s'entendent pas. Un jeudi, leur rancoeur éclate. "
 'Deux biscuits vont tout compliquer en créant un choc mystique. Le lendemain, '
 "Tess et Anna se retrouvent dans le corps l'une de l'autre...",
 'Astérix et Obélix : Mission Cléopâtre, Cléopâtre décide, pour défier '
 "l'Empereur romain Jules César, de construire en trois mois un palais "
 'somptueux en plein désert. Si elle y parvient, celui-ci devra concéder que '
 'le peuple é